In [1]:
import math
import numpy as np
import h5py
# import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
import json
import random
from collections import OrderedDict

# %matplotlib inline
# np.random.seed(0)

In [2]:
import cv2
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [9]:
EPOCHS = 10
BATCH_SIZE = 50
IMG_SIZE = 224
data_file = "dataset.txt"

In [4]:
def conv_layer(input_data,filter_size,stride_size,pad="SAME",name="conv",weight_name="W1",bias_name="b1"):
    print("creating layer :"+name)
    with tf.name_scope(name):
        W = tf.get_variable(weight_name, shape = filter_size, initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable(bias_name, shape = [filter_size[-1]], initializer = tf.constant_initializer(0.0))
        
        conv = tf.nn.conv2d(input_data, W, strides=stride_size, padding=pad)
        out = tf.nn.bias_add(conv, biases)
        act = tf.nn.relu(out)
        tf.summary.histogram("biases", biases)
        tf.summary.histogram("weights", W)
        tf.summary.histogram("activations", act)
        return act

In [5]:
def vggnet_v1(input_data):
    print("creating model")
    output=[]
    layer_input = input_data
    
    print("reading vgg16 json file")
    vgg = json.load(open("vgg16.json"),object_pairs_hook=OrderedDict)
   
    for index,layer in enumerate(vgg):
        if "conv" in layer:
            output = conv_layer(layer_input,vgg[layer]["weights"],vgg[layer]["stride"],vgg[layer]["pad"],layer,"weight"+str(index),"bias"+str(index))
        elif "pool" in layer:
            output = tf.nn.max_pool(layer_input, ksize=vgg[layer]["ksize"], strides=vgg[layer]["strides"], padding=vgg[layer]["padding"],name=layer)
        layer_input = output
    
    P3 = tf.contrib.layers.flatten(output)
    
    fc6 = tf.contrib.layers.fully_connected(P3,4096,biases_initializer=tf.constant_initializer(1.0), scope="fc6")
    tf.summary.histogram("fc6/relu", fc6)
    
    dropout1 = tf.layers.dropout(inputs=fc6, rate=0.5)
    
    fc7 = tf.contrib.layers.fully_connected(dropout1,4096,biases_initializer=tf.constant_initializer(1.0), scope="fc7")
    tf.summary.histogram("fc7/relu", fc7)
    
    dropout2 = tf.layers.dropout(inputs=fc7, rate=0.5)
    
    fc8 = tf.contrib.layers.fully_connected(dropout2,2, biases_initializer=tf.constant_initializer(1.0), activation_fn=None, scope="fc8")
    tf.summary.histogram("fc8", fc8)
    
    
    print("model completed.")
    ##NEED FULLY CONNECTED LAYER
    return fc8

In [ ]:
def vggnet_v2(input_data):
    
    # conv1_1
    conv1_1 = conv_layer(input_data,[3,3,3,64],[1, 1, 1, 1],"SAME","conv1_1","weight1")

    # conv1_2
    conv1_2 = conv_layer(conv1_1,[3,3,64,64],[1, 1, 1, 1],"SAME","conv1_2","weight2")
        
    # pool1
    pool1 = tf.nn.max_pool(conv1_2,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding="SAME",name='pool1')
        
    # conv2_1
    conv2_1 = conv_layer(pool1,[3, 3, 64, 128],[1, 1, 1, 1],"SAME","conv2_1","weight3")
        
    # conv2_2
    conv2_2 = conv_layer(conv2_1,[3,3,128,128],[1, 1, 1, 1],"SAME","conv2_2","weight4")

    # pool2
    pool2 = tf.nn.max_pool(conv2_2,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool2')

    # conv3_1
    conv3_1 = conv_layer(pool2,[3,3,128,256],[1, 1, 1, 1],"SAME","conv3_1","weight5")
    
    # conv3_2
    conv3_2 = conv_layer(conv3_1,[3,3,256,256],[1, 1, 1, 1],"SAME","conv3_2","weight6")
    
    # conv3_3
    conv3_3 = conv_layer(conv3_2,[3,3,256,256],[1, 1, 1, 1],"SAME","conv3_3","weight7")

    # pool3
    pool3 = tf.nn.max_pool(conv3_3,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool3')

    # conv4_1
    conv4_1 = conv_layer(pool3,[3, 3, 256, 512],[1, 1, 1, 1],"SAME","conv4_1","weight8")
        
    # conv4_2
    conv4_2 = conv_layer(conv4_1,[3, 3, 512, 512],[1, 1, 1, 1],"SAME","conv4_2","weight9")

    # conv4_3
    conv4_3 = conv_layer(conv4_2,[3, 3, 512, 512],[1, 1, 1, 1],"SAME","conv4_3","weight10")

    # pool4
    pool4 = tf.nn.max_pool(conv4_3,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool4')

    # conv5_1
    conv5_1 = conv_layer(pool4,[3, 3, 512, 512],[1, 1, 1, 1],"SAME","conv5_1","weight11")
   
    # conv5_2
    conv5_2 = conv_layer(conv5_1,[3, 3, 512, 512],[1, 1, 1, 1],"SAME","conv5_2","weight12")

    # conv5_3
    conv5_3 = conv_layer(conv5_2,[3, 3, 512, 512],[1, 1, 1, 1],"SAME","conv5_3","weight13")

    # pool5
    pool5 = tf.nn.max_pool(conv5_3, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool5')
    
    flatten = tf.contrib.layers.flatten(pool5)
    print("flatten")
    print(flatten)
    
    fc6 = tf.contrib.layers.fully_connected(flatten,4096,scope="fc6")
    tf.summary.histogram("fc6/relu", fc6)
    
#     dropout1 = tf.layers.dropout(inputs=fc6, rate=0.5)
    
    fc7 = tf.contrib.layers.fully_connected(fc6,4096,scope="fc7")
    tf.summary.histogram("fc7/relu", fc7)
    
#     dropout = tf.layers.dropout(inputs=fc7, rate=0.5)
    
    fc8 = tf.contrib.layers.fully_connected(fc7,2,activation_fn=None,scope="fc8")
    tf.summary.histogram("fc8", fc8)
    print("fc8:")
    print(fc8)
    print("model completed.")
    return fc8
    

In [ ]:
def vggnet_v3(images):
    
    # conv1_1
    with tf.name_scope('conv1_1') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv1_1 = tf.nn.relu(out, name=scope)
        

    # conv1_2
    with tf.name_scope('conv1_2') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv1_2 = tf.nn.relu(out, name=scope)
        

    # pool1
    pool1 = tf.nn.max_pool(conv1_2,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='SAME',
                           name='pool1')

    # conv2_1
    with tf.name_scope('conv2_1') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv2_1 = tf.nn.relu(out, name=scope)
        

    # conv2_2
    with tf.name_scope('conv2_2') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv2_2 = tf.nn.relu(out, name=scope)
        

    # pool2
    pool2 = tf.nn.max_pool(conv2_2,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='SAME',
                           name='pool2')

    # conv3_1
    with tf.name_scope('conv3_1') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv3_1 = tf.nn.relu(out, name=scope)
        

    # conv3_2
    with tf.name_scope('conv3_2') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv3_1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv3_2 = tf.nn.relu(out, name=scope)
        

    # conv3_3
    with tf.name_scope('conv3_3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv3_2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv3_3 = tf.nn.relu(out, name=scope)
        

    # pool3
    pool3 = tf.nn.max_pool(conv3_3,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='SAME',
                           name='pool3')

    # conv4_1
    with tf.name_scope('conv4_1') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool3, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv4_1 = tf.nn.relu(out, name=scope)
        

    # conv4_2
    with tf.name_scope('conv4_2') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv4_1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv4_2 = tf.nn.relu(out, name=scope)
        

    # conv4_3
    with tf.name_scope('conv4_3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv4_2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv4_3 = tf.nn.relu(out, name=scope)
        

    # pool4
    pool4 = tf.nn.max_pool(conv4_3,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='SAME',
                           name='pool4')

    # conv5_1
    with tf.name_scope('conv5_1') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool4, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv5_1 = tf.nn.relu(out, name=scope)
        

    # conv5_2
    with tf.name_scope('conv5_2') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv5_1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv5_2 = tf.nn.relu(out, name=scope)
        

    # conv5_3
    with tf.name_scope('conv5_3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv5_2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                             trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        conv5_3 = tf.nn.relu(out, name=scope)
        

    # pool5
    pool5 = tf.nn.max_pool(conv5_3,
                           ksize=[1, 2, 2, 1],
                           strides=[1, 2, 2, 1],
                           padding='SAME',
                           name='pool4')
    
    
    # fc1
    with tf.name_scope('fc1') as scope:
        shape = int(np.prod(pool5.get_shape()[1:]))
        fc1w = tf.Variable(tf.truncated_normal([shape, 4096],
                                                     dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
        fc1b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
                             trainable=True, name='biases')
        pool5_flat = tf.reshape(pool5, [-1, shape])
        fc1l = tf.nn.bias_add(tf.matmul(pool5_flat, fc1w), fc1b)
        fc1 = tf.nn.relu(fc1l)


    # fc2
    with tf.name_scope('fc2') as scope:
        fc2w = tf.Variable(tf.truncated_normal([4096, 4096],
                                                     dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
        fc2b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
                             trainable=True, name='biases')
        fc2l = tf.nn.bias_add(tf.matmul(fc1, fc2w), fc2b)
        fc2 = tf.nn.relu(fc2l)


    # fc3
    with tf.name_scope('fc3') as scope:
        fc3w = tf.Variable(tf.truncated_normal([4096, 2],
                                                     dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
        fc3b = tf.Variable(tf.constant(1.0, shape=[2], dtype=tf.float32),
                             trainable=True, name='biases')
        fc3l = tf.nn.bias_add(tf.matmul(fc2, fc3w), fc3b)
#         fc3 = tf.nn.relu(fc3l)

    return fc3l

In [6]:
def prepare_data(data):
    newData=[]
    labels=[]
    print("preparing data....")
    for sample in data:
        img_path,label = sample.strip().split(" ")
        img = Image.open(img_path)
        img = img.resize((IMG_SIZE,IMG_SIZE))
        img = np.array(img)
        img = img/255
        newData.append(img)
        if "cat" in img_path:
            labels.append(np.array([1,0]))
        elif "dog" in img_path:
            labels.append(np.array([0,1]))
    print("preparing data completed.")
    return np.array(newData),np.array(labels)

In [7]:
f = open(data_file,"r")
data = f.readlines()
random.shuffle(data)
print(len(data))
train_data = data[:20000]
train_X, train_Y = prepare_data(train_data)
val_data = data[20000:]
val_X, val_Y = prepare_data(val_data)

25000
preparing data....
preparing data completed.
preparing data....
preparing data completed.


In [ ]:
tf.reset_default_graph()
start_learning_rate = 0.01
batch = tf.Variable(0, trainable=False)
train_size = 20000
val_size = 5000
learning_rate = tf.train.exponential_decay(
  0.001,                # Base learning rate.
  batch * BATCH_SIZE,  # Current index into the dataset.
  train_size,          # Decay step.
  0.1,                # Decay rate.
  staircase=True)


with tf.name_scope('inputs'):
    X = tf.placeholder(tf.float32, shape=(None,IMG_SIZE,IMG_SIZE , 3),name="X")
    tf.summary.image('input', X, BATCH_SIZE)
    Y = tf.placeholder(tf.float32,shape=(None,2),name="labels")
    
logits = vggnet_v1(X)
with tf.name_scope("loss"):    
    train_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="loss")
    tf.summary.scalar("loss", train_loss)

with tf.name_scope("train"):
    #optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(train_loss)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(train_loss,global_step=batch)
print("optimizer set")

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)
print("accuracy set")

merged_summary = tf.summary.merge_all()

# init = 
# print("initializing the variables")

with tf.Session() as sess:
    print("session created.")
    sess.run(tf.global_variables_initializer())
    
    print("writing tensorboard")
    writer = tf.summary.FileWriter("/tmp/tensorboard/cat-dog-vgg2")
    writer.add_graph(sess.graph)
    print("writer added to graph")
    
    for epoch in range(EPOCHS):
        
        print("processing epoch "+str(epoch))
        print("total number of batches in 1 epoch: "+ str(len(train_data)/BATCH_SIZE))
        
        for i in range(0,train_size,BATCH_SIZE): 
            #print(i)
            print("Batch:" + str((i/BATCH_SIZE)+1))
            
            #minibatch_X, minibatch_Y = prepare_data(train_data[i:i+BATCH_SIZE])
            #sess.run(optimizer,feed_dict={X:minibatch_X,Y:minibatch_Y})
            minibatch_X = train_X[i : i + BATCH_SIZE]
            minibatch_Y = train_Y[i : i + BATCH_SIZE]
            _ , temp_loss, temp_accuracy = sess.run([optimizer,train_loss,accuracy],feed_dict={X:minibatch_X,Y:minibatch_Y})
            
            if i % (BATCH_SIZE*10) == 0 and i >200:  # Record summaries and test-set accuracy
                summary = sess.run(merged_summary, feed_dict={X:minibatch_X,Y:minibatch_Y})
                writer.add_summary(summary, i + epoch*len(train_data))
            
            #writer.add_summary(summary, i)
            print("temp_loss: "+str(temp_loss))
            
            if i % (BATCH_SIZE*50) == 0 and i>=1000:
                print("validating...")
                val_loss = 0
                val_accuracy = 0
                val_batch_size = 50
                val_batches = val_size/val_batch_size
                for v in range(0,val_size,val_batch_size):
                    #val_x,val_y = prepare_data(val_data[v:v+BATCH_SIZE])
                    val_minibatch_x = val_X[v : v + val_batch_size]
                    val_minibatch_y = val_Y[v : v + val_batch_size]
                    temp_loss, temp_accuracy = sess.run([train_loss,accuracy],feed_dict={X:val_minibatch_x,Y:val_minibatch_y})
                    val_accuracy+=temp_accuracy
                    val_loss+=temp_loss
                print("validation loss:")
                print(val_loss/val_batches)
                print("validation accuracy: ")
                print(val_accuracy/val_batches)
#             print("temp_accuracy: "+str(temp_accuracy))
#     train_accuracy = accuracy.eval({X: train_data, Y: train_labels})
#     val_data, val_labels = prepare_data(val_data)
#     val_accuracy = accuracy.eval({X: val_data, Y: val_labels})
# #     print("Train Accuracy:", train_accuracy)
#     print("Validation Accuracy:", val_accuracy)

creating model
reading vgg16 json file
creating layer :conv1_1
creating layer :conv1_2
creating layer :conv2_1
creating layer :conv2_2
creating layer :conv3_1
creating layer :conv3_2
creating layer :conv3_3
creating layer :conv4_1
creating layer :conv4_2
creating layer :conv4_3
creating layer :conv5_1
creating layer :conv5_2
creating layer :conv5_3
model completed.
optimizer set
accuracy set
session created.
writing tensorboard
writer added to graph
processing epoch 0
total number of batches in 1 epoch: 400.0
Batch:1.0
temp_loss: 1.01737
Batch:2.0
temp_loss: 384.513
Batch:3.0
temp_loss: 87.2746
Batch:4.0
temp_loss: 71.1724
Batch:5.0
temp_loss: 33.8528
Batch:6.0
temp_loss: 5.47547
Batch:7.0
temp_loss: 0.721761
Batch:8.0
temp_loss: 1.7492
Batch:9.0
temp_loss: 10.3293
Batch:10.0
temp_loss: 3.96727
Batch:11.0
temp_loss: 0.852472
Batch:12.0
temp_loss: 4.00442
Batch:13.0
temp_loss: 2.38266
Batch:14.0
temp_loss: 2.71777
Batch:15.0
temp_loss: 2.71458
Batch:16.0
temp_loss: 2.14912
Batch:17.0
te